In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import irfft, rfft, rfftfreq
import os
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

In [2]:
files = []
directory = 'static_signs_eeg_seed/'  #директория с файлами статических признаков для каждого канала ээг
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        files.append(f)

len(files)

32

In [14]:
files[13][22:-4]

'fz'

In [3]:
#seed
labels = scipy.io.loadmat('SEED_EEG/Preprocessed_EEG/label.mat')['label']
labels = np.array([labels[0]]*45).reshape(1,675)[0]

In [4]:
names= []
for file in files:
    names.append(file[25:-8])

In [6]:
def calculate(data,eeg,join=True):
    result_table = pd.DataFrame()
    result_table['criterion'] = ['accuracy_score', 'f1_score', 'precision_score', 'recall_score']
    for file in files:
        if join:
            data_all = data.join(pd.read_csv(file,index_col=0),rsuffix=eeg[-1][22:-4])
        else:
            data_all = pd.read_csv(file,index_col=0)
            
        # Разделение выборки для дикторозависимых моделей
        X_train, X_test, y_train, y_test = train_test_split(data_all, labels, test_size=0.25, random_state=249)
        
        # Разделение выборки для дикторонезависимых моделей
#         X_train = data_all[:506]
#         X_test = data_all[506:]
#         y_train = labels[:506]
#         y_test = labels[506:]
        
        tree = RandomForestClassifier(criterion='entropy',
             n_estimators=15,
             max_depth=8,
             min_samples_leaf= 10,
             min_samples_split = 20)
        clf = AdaBoostClassifier(base_estimator=tree,n_estimators=10, random_state=0)
        clf.fit(X_train, y_train)
        results = clf.predict(X_test)
        
        result_table[file] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
        
    if join:
        result_table = result_table.drop(columns=eeg)
    return result_table.T.iloc[1:,:].rename(columns={0:'acc',1:'f1',2:'precision',3:'recall'}).sort_values(by='acc',ascending=False)

def next_step(data,eeg,prev):
    print(eeg)
    for elem in prev.index:
        eeg.append(elem)
        eeg_name = [i[22:-4] for i in eeg]
        all_check[' '.join(eeg_name)] = prev.loc[[elem]].to_numpy()[0]
        if len(eeg) < 7:
            data_all = data.join(pd.read_csv(elem,index_col=0),rsuffix=elem[22:-4])
            res = calculate(data_all,eeg)
            next_step(data_all,eeg,res[:3])
        eeg.pop()
    

In [7]:
all_check = pd.DataFrame()
all_check['criterion'] = ['accuracy_score', 'f1_score', 'precision_score', 'recall_score']
start = calculate([],[],False)
for elem in start.index[:10]:
    all_check[elem[22:-4]] = start.loc[[elem]].to_numpy()[0]
    data = pd.read_csv(elem,index_col=0)
    queue = [elem]
    res = calculate(data,queue)
    next_step(data,queue,res[:3])
    all_check.T.iloc[1:,:].rename(columns={0:'acc',1:'f1',2:'precision',3:'recall'}).to_csv('eeg_group_seed_independent' + elem[22:-4] + '.csv')



['static_signs_eeg_seed/t7.csv']
['static_signs_eeg_seed/t7.csv', 'static_signs_eeg_seed/oz.csv']
['static_signs_eeg_seed/t7.csv', 'static_signs_eeg_seed/oz.csv', 'static_signs_eeg_seed/o2.csv']
['static_signs_eeg_seed/t7.csv', 'static_signs_eeg_seed/oz.csv', 'static_signs_eeg_seed/o2.csv', 'static_signs_eeg_seed/с3.csv']
['static_signs_eeg_seed/t7.csv', 'static_signs_eeg_seed/oz.csv', 'static_signs_eeg_seed/o2.csv', 'static_signs_eeg_seed/с3.csv', 'static_signs_eeg_seed/po4.csv']
['static_signs_eeg_seed/t7.csv', 'static_signs_eeg_seed/oz.csv', 'static_signs_eeg_seed/o2.csv', 'static_signs_eeg_seed/с3.csv', 'static_signs_eeg_seed/po4.csv', 'static_signs_eeg_seed/с4.csv']
['static_signs_eeg_seed/t7.csv', 'static_signs_eeg_seed/oz.csv', 'static_signs_eeg_seed/o2.csv', 'static_signs_eeg_seed/с3.csv', 'static_signs_eeg_seed/po4.csv', 'static_signs_eeg_seed/f7.csv']
['static_signs_eeg_seed/t7.csv', 'static_signs_eeg_seed/oz.csv', 'static_signs_eeg_seed/o2.csv', 'static_signs_eeg_seed/с3.csv

## Результаты

In [2]:
seed = pd.read_csv('eeg_group_seed.csv',index_col=0).drop_duplicates()
seed.sort_values(by='acc',ascending=False).head(10)

,acc,f1,precision,recall
fс5 t7 p8 p3 af3 fс6,0.857988,0.857883,0.859705,0.857988
cp5 p8 t7 po3 fp1 t8 fс2,0.857988,0.859113,0.863975,0.857988
f8 p8 af3 t8 с3 fс5 cp6,0.857988,0.858656,0.861990,0.857988
fp2 cp5 t8 po3 f4 t7 p8,0.857988,0.859246,0.863426,0.857988
po4 t8 fp1 t7 p3 fс5 fс1,0.852071,0.854264,0.862469,0.852071
f8 p8 t7 fp2 t8 p3 af3,0.852071,0.853953,0.860989,0.852071
cp5 t7 fp2 po3 af3 p8 t8,0.852071,0.853367,0.860510,0.852071
cp5 t7 fp2 fс5 f8 p8 t8,0.852071,0.852579,0.855893,0.852071
fp2 t8 o1 fс5 p8 t7 f8,0.852071,0.853723,0.858141,0.852071
cp5 t8 p8 fp1 fс6 t7 f4,0.852071,0.854128,0.860836,0.852071


In [3]:
seed = pd.read_csv('eeg_group_seed_independent.csv',index_col=0).drop_duplicates()
seed.sort_values(by='acc',ascending=False).head(10)

,acc,f1,precision,recall
t8 p7 oz p8 o2 f3 fz,0.798817,0.798426,0.800222,0.798817
p7 p8 с4 o2 t8 o1 po4,0.798817,0.797930,0.799266,0.798817
o1 t8 f3 fс2 fp1 f7 cp5,0.792899,0.792039,0.791508,0.792899
fс6 p8 o2 fz cp5 t8 f7,0.786982,0.786286,0.786600,0.786982
cp5 o1 t8 cp1 f3 o2 cz,0.786982,0.785667,0.784901,0.786982
fс6 po4 cp5 t8 o2 fс2,0.781065,0.779729,0.779409,0.781065
p8 o2 fp1 fс2 o1 t8 cp1,0.781065,0.779156,0.785559,0.781065
p8 o1 fp1 fp2 t8,0.781065,0.781708,0.782615,0.781065
p7 p8 t8 po3 f3 oz af4,0.781065,0.779744,0.780828,0.781065
p7 p8 t8 po4 fp2 oz o1,0.781065,0.781168,0.782225,0.781065
